In [1]:
# df =pd.read_csv('raw_4M_20m_pt_data_with_sprade.csv')
# #convert all negative values to positive
# df['buy_highest_loss_spread'] = df['buy_highest_loss_spread'].abs()
# df['sell_highest_loss_spread'] = df['sell_highest_loss_spread'].abs()
# df.to_csv('raw_4M_20m_pt_data_with_sprade.csv', index=False)

In [2]:
# Import Libraries
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import optuna
import pandas as pd
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn 
import tqdm
from torch.utils.data import DataLoader,Dataset
from sklearn.model_selection import train_test_split

from torch.utils.data.sampler import Sampler


data = pd.read_csv('raw_4M_20m_pt_data_with_sprade.csv')
#train test val split do not randomize the data because it is a time series forex data
train = data[:int(0.8*len(data))]
val = data[int(0.8*len(data)):int(0.9*len(data))]
test = data[int(0.9*len(data)):]


# in the train data define the x and y(train and target)
x_train = train['Close']
y_train_1 = train['buy_open_holding']
y_train_2 = train['sell_open_holding']
y_train_3 = train['buy_spread']
y_train_4 = train['sell_spread']
#take positive true vale of buy_highest_loss_spread
y_train_5 = train['buy_highest_loss_spread'].abs()
y_train_6 = train['sell_highest_loss_spread'].abs()
# each loop i will pass preious 24 candales of data and has to predict theif to buy or sell or do nothing if buy or sell then how long to hold or when to close the trade





class ForexDataset(Dataset):
    def __init__(self, data, sequence_length=256):
        self.data = data
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
            
        # Extract sequences of length 'sequence_length' for input features
        x_sequence = torch.tensor(self.data['Close'].values[idx:idx + self.sequence_length], dtype=torch.float32)

        # Target labels (buy, sell, hold)
        y_buy = torch.tensor(self.data['buy_open_holding'].values[idx + self.sequence_length], dtype=torch.float32)
        # print('buy ',y_buy)        

        y_sell = torch.tensor(self.data['sell_open_holding'].values[idx + self.sequence_length], dtype=torch.float32)
    
        # y_buy_spread = torch.tensor(self.data['buy_spread'].values[idx + self.sequence_length], dtype=torch.float32)
        # y_sell_spread = torch.tensor(self.data['sell_spread'].values[idx + self.sequence_length], dtype=torch.float32)
        # y_buy_highest_loss_spread = torch.tensor(self.data['buy_highest_loss_spread'].values[idx + self.sequence_length], dtype=torch.float32)
        # y_sell_highest_loss_spread = torch.tensor(self.data['sell_highest_loss_spread'].values[idx + self.sequence_length], dtype=torch.float32)
        # y_sequence = torch.stack([y_buy, y_sell, y_buy_spread, y_sell_spread, y_buy_highest_loss_spread, y_sell_highest_loss_spread])

        y_sequence = torch.stack([y_buy, y_sell])


        return x_sequence, y_sequence
    


class PositiveSampleRatioSampler(Sampler):
    def __init__(self, dataset, batch_size, positive_ratio=0.5):
        self.dataset = dataset
        self.batch_size = batch_size
        self.positive_ratio = positive_ratio

    def __iter__(self): 
        positive_idx = []
        negative_idx = []

        # Iterate through all samples in the dataset
        for idx in range(len(self.dataset)):
            x, y = self.dataset[idx]

            buy_y = y[0]
            sell_y = y[1]
            # buy_spread_y = y[2]
            # sell_spread_y = y[3]
            # buy_highest_loss_spread_y = y[4]
            # sell_highest_loss_spread_y = y[5]

            # If the target label is 'buy' or 'sell', add the sample index to the positive list
            if buy_y > 0 or sell_y > 0:
                positive_idx.append(idx)
            else:
                negative_idx.append(idx)

        # Calculate the number of positive samples to include in each batch
        num_positive = int(self.batch_size * self.positive_ratio)
        num_negative = self.batch_size - num_positive

        # Randomly sample positive indices
        positive_samples = torch.randperm(len(positive_idx))[:num_positive].tolist()
        # Randomly sample negative indices
        negative_samples = torch.randperm(len(negative_idx))[:num_negative].tolist()

        # Combine positive and negative samples
        batch_indices = positive_samples + negative_samples

        return iter(batch_indices)

    def __len__(self):
        return len(self.dataset)        


LR = 0.00022528492132355385
BATCH_SIZE = 64
EPOCHS = 300


# Assuming 'train', 'val', and 'test' are your data splits
train_dataset = ForexDataset(train)
val_dataset = ForexDataset(val)
test_dataset = ForexDataset(test)

# Define batch size
batch_size = BATCH_SIZE







# Create PositiveSampleRatioSampler for training data
train_sampler = PositiveSampleRatioSampler(train_dataset, batch_size, positive_ratio=0.5)

# Create DataLoader instances using the custom sampler
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)












# Create DataLoader instances
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Define Optimizer and Loss Function
writer = SummaryWriter()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



# model = nn.Sequential(
#     nn.Linear(in_features=256, out_features=1024),
#     nn.BatchNorm1d(1024),   # Batch Normalization
#     nn.ReLU(),
#     nn.Dropout(p=0.2),
#     nn.Linear(in_features=1024, out_features=512),
#     nn.BatchNorm1d(512),    # Batch Normalization
#     nn.ReLU(),
#     nn.Dropout(p=0.2),
#     nn.Linear(in_features=512, out_features=256),
#     nn.BatchNorm1d(256),    # Batch Normalization
#     nn.ReLU(),
#     nn.Dropout(p=0.2),
#     nn.Linear(in_features=256, out_features=2)
# )

# model = nn.Sequential(
#     nn.Conv1d(in_channels=256, out_channels=1024, kernel_size=3),
#     nn.BatchNorm1d(1024),   # Batch Normalization
#     nn.ReLU(),
#     nn.Dropout(p=0.2),
#     nn.Conv1d(in_channels=1024, out_channels=512, kernel_size=3),
#     nn.BatchNorm1d(512),    # Batch Normalization
#     nn.ReLU(),
#     nn.Dropout(p=0.2),
#     nn.Conv1d(in_channels=512, out_channels=256, kernel_size=3),
#     nn.BatchNorm1d(256),    # Batch Normalization
#     nn.ReLU(),
#     nn.Dropout(p=0.2),
#     nn.Flatten(),
#     nn.Linear(in_features=256, out_features=2)
# )


model = nn.Sequential(
    nn.Conv1d(in_channels=1, out_channels=1024, kernel_size=3),
    nn.BatchNorm1d(1024),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Conv1d(in_channels=1024, out_channels=512, kernel_size=3),
    nn.BatchNorm1d(512),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Conv1d(in_channels=512, out_channels=256, kernel_size=3),
    nn.BatchNorm1d(256),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Flatten(),
    nn.Linear(in_features=64000, out_features=2)
)


# model = nn.Sequential(
#     nn.Conv1d(in_channels=1, out_channels=1024, kernel_size=3),
#     nn.BatchNorm1d(1024),   # Batch Normalization
#     nn.ReLU(),
#     nn.Dropout(p=0.2),
#     nn.Conv1d(in_channels=1024, out_channels=512, kernel_size=3),
#     nn.BatchNorm1d(512),    # Batch Normalization
#     nn.ReLU(),
#     nn.Dropout(p=0.2),
#     nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3),
#     nn.BatchNorm1d(1),    # Batch Normalization
#     nn.ReLU(),
#     nn.Dropout(p=0.2),
#     nn.Flatten(),
#     nn.Linear(in_features=250, out_features=2)




    
# )


model = model.to(device)







# #load the model
model.load_state_dict(torch.load('model_pytorch_paralal_6_conv_45.pt'))


optimizer = optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.MSELoss(reduction='mean')





c:\Users\rifat\miniconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# model = nn.Sequential(
#     nn.Conv1d(in_channels=1, out_channels=1024, kernel_size=3),
#     nn.BatchNorm1d(1024),   # Batch Normalization
#     nn.ReLU(),
#     nn.Dropout(p=0.2),
#     nn.Conv1d(in_channels=1024, out_channels=512, kernel_size=3),
#     nn.BatchNorm1d(512),    # Batch Normalization
#     nn.ReLU(),
#     nn.Dropout(p=0.2),
#     nn.Conv1d(in_channels=512, out_channels=256, kernel_size=3),
#     nn.BatchNorm1d(256),    # Batch Normalization
#     nn.ReLU(),
#     nn.Dropout(p=0.2),
#     nn.Flatten(),
#     nn.Linear(in_features=64000, out_features=2)
# )


# x = torch.randn(1, 1, 256)
# print(x.shape)
# print(model(x).shape)


In [4]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# from torchinfo import summary
# summary(model, input_size=(1,1, 256), device=device)

In [6]:
# model = nn.Sequential(
#     nn.Conv1d(in_channels=1, out_channels=1024, kernel_size=3),
#     nn.BatchNorm1d(1024),   # Batch Normalization
#     nn.ReLU(),
#     nn.Dropout(p=0.2),
#     nn.Conv1d(in_channels=1024, out_channels=512, kernel_size=3),
#     nn.BatchNorm1d(512),    # Batch Normalization
#     nn.ReLU(),
#     nn.Dropout(p=0.2),
#     nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3),
#     nn.BatchNorm1d(1),    # Batch Normalization
#     nn.ReLU(),
#     nn.Dropout(p=0.2),
#     nn.Flatten(),
#     nn.Linear(in_features=250, out_features=2)




    
# )


# x = torch.randn(1, 1, 256)
# print(x.shape)
# print(model(x).shape)


In [7]:
# from torchinfo import summary
# summary(model, input_size=(1,1, 256), device=device)

In [8]:
with tqdm.tqdm(total=EPOCHS) as epoch_pbar:
    for epoch in range(EPOCHS):
        # Training
        model.train()
        train_losses = []
        for x_train_batch, y_train_batch in train_loader:

            x_train_batch = x_train_batch.unsqueeze(1)            
            x_train_batch = x_train_batch.to(device)
            y_train_batch = y_train_batch.to(device)

            # Forward Propagation
            y_train_pred = model(x_train_batch)

            # Calculate Loss
            train_loss = loss_fn(y_train_pred, y_train_batch)








            train_losses.append(train_loss.item())





            # Backpropagation
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        val_losses = []
        for x_val_batch, y_val_batch in val_loader:
            x_val_batch = x_val_batch.unsqueeze(1)
            x_val_batch = x_val_batch.to(device)
            y_val_batch = y_val_batch.to(device)

            # Forward Propagation
            y_val_pred = model(x_val_batch)

            # Calculate Loss
            val_loss = loss_fn(y_val_pred ,y_val_batch)
            val_losses.append(val_loss.item())

        # Calculate Average Loss
        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)

        # Print Epoch, Loss, and Accuracy
        print(f'Epoch: {epoch + 1}/{EPOCHS}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
        model_path = f'model_pytorch_paralal_6_conv_{45+epoch + 1}.pt'
        torch.save(model.state_dict(), model_path)

        # Log to TensorBoard
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)

        epoch_pbar.update(1)
        





  0%|          | 1/300 [04:40<23:19:35, 280.85s/it]

Epoch: 1/300, Train Loss: 89.0736, Val Loss: 113.4933


  1%|          | 2/300 [09:26<23:27:49, 283.46s/it]

Epoch: 2/300, Train Loss: 97.3907, Val Loss: 117.4215


  1%|          | 3/300 [14:19<23:45:23, 287.96s/it]

...

Epoch: 189/300, Train Loss: 75.2606, Val Loss: 83.7959


 63%|██████▎   | 190/300 [15:33:39<8:20:01, 272.74s/it]

Epoch: 190/300, Train Loss: 76.6138, Val Loss: 85.8925


 64%|██████▎   | 191/300 [15:38:14<8:17:03, 273.61s/it]

Epoch: 191/300, Train Loss: 80.4170, Val Loss: 83.5922


In [8]:
with tqdm.tqdm(total=EPOCHS) as epoch_pbar:
    for epoch in range(EPOCHS):
        # Training
        model.train()
        train_losses = []
        for x_train_batch, y_train_batch in train_loader:

            x_train_batch = x_train_batch.unsqueeze(1)            
            x_train_batch = x_train_batch.to(device)
            y_train_batch = y_train_batch.to(device)

            # Forward Propagation
            y_train_pred = model(x_train_batch)

            # Calculate Loss
            train_loss = loss_fn(y_train_pred, y_train_batch)








            train_losses.append(train_loss.item())





            # Backpropagation
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        val_losses = []
        for x_val_batch, y_val_batch in val_loader:
            x_val_batch = x_val_batch.unsqueeze(1)
            x_val_batch = x_val_batch.to(device)
            y_val_batch = y_val_batch.to(device)

            # Forward Propagation
            y_val_pred = model(x_val_batch)

            # Calculate Loss
            val_loss = loss_fn(y_val_pred ,y_val_batch)
            val_losses.append(val_loss.item())

        # Calculate Average Loss
        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)

        # Print Epoch, Loss, and Accuracy
        print(f'Epoch: {epoch + 1}/{EPOCHS}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
        model_path = f'model_pytorch_paralal_6_conv_{45+epoch + 1}.pt'
        torch.save(model.state_dict(), model_path)

        # Log to TensorBoard
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)

        epoch_pbar.update(1)
        





  0%|          | 1/300 [05:08<25:35:04, 308.04s/it]

Epoch: 1/300, Train Loss: 82.1170, Val Loss: 84.2778


  1%|          | 2/300 [10:15<25:29:08, 307.88s/it]

Epoch: 2/300, Train Loss: 82.7792, Val Loss: 82.0281


  1%|          | 3/300 [15:26<25:30:49, 309.26s/it]

...

Epoch: 43/300, Train Loss: 79.3025, Val Loss: 79.0868


 15%|█▍        | 44/300 [10:01:37<31:44:41, 446.41s/it]

Epoch: 44/300, Train Loss: 74.7340, Val Loss: 81.7882


 15%|█▌        | 45/300 [10:08:33<30:59:22, 437.50s/it]

Epoch: 45/300, Train Loss: 78.8469, Val Loss: 84.9441


In [ ]:
model = nn.Sequential(
    nn.Conv1d


In [ ]:
#save the model
torch.save(model.state_dict(), 'model_pytorch_conv_1.01.pt')

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import tqdm

# Assuming you have defined your model, optimizer, and loss function before this code snippet

# Define the number of epochs
EPOCHS = 10

# Initialize TensorBoard writer
writer = SummaryWriter()

# Training loop
with tqdm.tqdm(total=EPOCHS) as epoch_pbar:
    for epoch in range(EPOCHS):
        # Training
        model.train()
        train_losses = []

        for x_train_batch, y_train_batch in train_loader:
            x_train_batch, y_train_batch = x_train_batch.to(device), y_train_batch.to(device)

            # Forward Propagation
            y_train_pred = model(x_train_batch)

            # Calculate Loss
            train_loss = loss_fn(y_train_pred, y_train_batch)  # No need for torch.argmax here
            train_losses.append(train_loss.item())

            # Backpropagation
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        val_losses = []

        with torch.no_grad():
            for x_val_batch, y_val_batch in val_loader:
                x_val_batch, y_val_batch = x_val_batch.to(device), y_val_batch.to(device)

                # Forward Propagation
                y_val_pred = model(x_val_batch)

                # Calculate Loss
                val_loss = loss_fn(y_val_pred, y_val_batch)  # No need for torch.argmax here
                val_losses.append(val_loss.item())

        # Calculate Average Loss
        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)

        # Print Epoch, Loss, and Accuracy
        print(f'Epoch: {epoch + 1}/{EPOCHS}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

        # Log to TensorBoard
        writer.add_scalar('Loss/train', train_loss, epoch + 1)  # Use epoch + 1 to start from 1
        writer.add_scalar('Loss/val', val_loss, epoch + 1)  # Use epoch + 1 to start from 1

        # Save the model checkpoint (adjust the path accordingly)
        checkpoint_path = f'model_checkpoint_epoch_{epoch + 1}.pt'
        torch.save(model.state_dict(), checkpoint_path)

        epoch_pbar.update(1)


In [ ]:
#save the model
torch.save(model.state_dict(), 'model_pytorch_2.pt')